# 03.4 - Missing Value Handling and Data Cleaning

This notebook performs comprehensive missing value analysis and implements sophisticated, domain-aware imputation strategies tailored to each feature type.

**Key Features:**
- **Track-specific qualifying imputation**: Uses previous race at same track, preserves structural missingness
- **Same-race imputation**: Respects circuit, conditions, day (no forward fill across races)
- **Grid constraints**: Enforces Q2/Q3 structural missingness (eliminated drivers don't have times)
- **Enhanced logging**: Shows which imputation method succeeded at each step
- **Final cleanup**: Removes unnecessary columns after imputation

**Principles:**
- Use the most suitable method for each feature type (non-negotiable, no matter how complex)
- Preserve temporal relationships and group structures
- Avoid data leakage in imputation
- Preserve legitimate structural missingness (Q2/Q3 for eliminated drivers)
- Document rationale for each method

**Input:** `data/processed/master_races_refined.csv` (from 03.7)  
**Output:** Cleaned dataset with all missing values appropriately handled and unnecessary columns removed


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set up paths
# Get project root (works whether running from notebooks/ or F1/ folder)
PROJECT_ROOT = Path().resolve()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent

PROCESSED_ROOT = PROJECT_ROOT / "data" / "processed"

# Load refined dataset from 03.7
refined_file = PROCESSED_ROOT / "master_races_refined.csv"
if not refined_file.exists():
    print(f"ERROR: {refined_file} not found!")
    print("Please run notebook 03.7_feature_refinement.ipynb first.")
    raise FileNotFoundError(f"Required file not found: {refined_file}")

print(f"Loading refined dataset from: {refined_file}")
master = pd.read_csv(refined_file)
master['date'] = pd.to_datetime(master['date'], errors='coerce')

# Sort by date to ensure proper temporal ordering
master = master.sort_values(['year', 'round', 'date']).reset_index(drop=True)

print(f"✓ Loaded successfully: {master.shape[0]:,} rows × {master.shape[1]} columns")
print(f"Date range: {master['date'].min()} to {master['date'].max()}")

# Set plotting style
sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)


Loading refined dataset from: C:\Users\erikv\Downloads\F1\data\processed\master_races_refined.csv
✓ Loaded successfully: 12,358 rows × 100 columns
Date range: 1994-03-27 00:00:00 to 2024-12-08 00:00:00


## 1. Comprehensive Missing Data Analysis

First, we need to understand the nature and extent of missing data in our refined dataset.


MISSING DATA ANALYSIS

Total columns: 100
Columns with missing data: 48
Total missing values: 252,341

MISSING DATA BY CATEGORY

1. Placeholder Features (100% missing - to be removed):
   - lap_time_variance: 100.0% missing
   - throttle_variance: 100.0% missing
   - overtake_attempts: 100.0% missing
   - avg_pit_stops: 100.0% missing

2. Sprint Features (structural missingness - sprint races only 2021+):
   - sprint_results_resultId: 97.1% missing
   - sprint_results_constructorId: 97.1% missing
   - sprint_results_number: 97.1% missing
   - sprint_results_grid: 97.1% missing
   - sprint_results_position: 97.1% missing
   - sprint_results_positionText: 97.1% missing
   - sprint_results_positionOrder: 97.1% missing
   - sprint_results_points: 97.1% missing
   - sprint_results_laps: 97.1% missing
   - sprint_results_time: 97.1% missing
   - sprint_results_milliseconds: 97.1% missing
   - sprint_results_fastestLap: 97.1% missing
   - sprint_results_fastestLapTime: 97.1% missing
   - spri

<Figure size 1200x1440 with 0 Axes>

In [40]:
# Comprehensive missing data analysis
missing_stats = []
for col in master.columns:
    missing_count = master[col].isnull().sum()
    missing_pct = (missing_count / len(master)) * 100
    dtype = master[col].dtype
    missing_stats.append({
        'column': col,
        'missing_count': missing_count,
        'missing_pct': missing_pct,
        'dtype': str(dtype)
    })

missing_df = pd.DataFrame(missing_stats).sort_values('missing_pct', ascending=False)

print("="*70)
print("MISSING DATA ANALYSIS")
print("="*70)
print(f"\nTotal columns: {len(master.columns)}")
print(f"Columns with missing data: {(missing_df['missing_pct'] > 0).sum()}")
print(f"Total missing values: {missing_df['missing_count'].sum():,}")

# Categorize missing patterns
print("\n" + "="*70)
print("MISSING DATA BY CATEGORY")
print("="*70)

# 1. Placeholder features (100% missing - FastF1 data not extracted)
placeholder_cols = [col for col in master.columns if any(x in col.lower() for x in 
    ['lap_time_variance', 'throttle_variance', 'overtake_attempts', 'avg_pit_stops'])]
if placeholder_cols:
    print(f"\n1. Placeholder Features (100% missing - to be removed):")
    for col in placeholder_cols:
        pct = missing_df[missing_df['column'] == col]['missing_pct'].values[0]
        print(f"   - {col}: {pct:.1f}% missing")

# 2. Sprint features (structural missingness - only 2021+)
sprint_cols = [col for col in master.columns if 'sprint' in col.lower()]
if sprint_cols:
    print(f"\n2. Sprint Features (structural missingness - sprint races only 2021+):")
    for col in sprint_cols:
        pct = missing_df[missing_df['column'] == col]['missing_pct'].values[0]
        print(f"   - {col}: {pct:.1f}% missing")

# 3. Qualifying features (historical data gaps)
qualifying_cols = [col for col in master.columns if any(x in col.lower() for x in 
    ['qualify', 'q1', 'q2', 'q3'])]
if qualifying_cols:
    print(f"\n3. Qualifying Features (historical data gaps):")
    for col in qualifying_cols:
        pct = missing_df[missing_df['column'] == col]['missing_pct'].values[0]
        print(f"   - {col}: {pct:.1f}% missing")

# 4. Rolling/derived features (should be minimal)
rolling_cols = [col for col in master.columns if any(x in col for x in 
    ['_last_', '_avg_', '_rate_', '_PRE_RACE', '_at_circuit', '_trend_'])]
rolling_missing = missing_df[missing_df['column'].isin(rolling_cols) & (missing_df['missing_pct'] > 0)]
if len(rolling_missing) > 0:
    print(f"\n4. Rolling/Derived Features (should be minimal):")
    for _, row in rolling_missing.head(10).iterrows():
        print(f"   - {row['column']}: {row['missing_pct']:.2f}% missing")
else:
    print(f"\n4. Rolling/Derived Features: ✓ No missing values")

# 5. Other features
other_cols = [col for col in master.columns if col not in 
    placeholder_cols + sprint_cols + qualifying_cols + rolling_cols]
other_missing = missing_df[missing_df['column'].isin(other_cols) & (missing_df['missing_pct'] > 0)]
if len(other_missing) > 0:
    print(f"\n5. Other Features with Missing Values:")
    for _, row in other_missing.head(10).iterrows():
        print(f"   - {row['column']}: {row['missing_pct']:.2f}% missing")

# Visualize missing patterns
if (missing_df['missing_pct'] > 0).sum() > 0:
    plt.figure(figsize=(12, max(6, (missing_df['missing_pct'] > 0).sum() * 0.3)))
    missing_plot = missing_df[missing_df['missing_pct'] > 0].head(30)
    missing_plot.plot(x='column', y='missing_pct', kind='barh', legend=False)
    plt.title('Missing Data Percentage by Column (Top 30)', fontsize=14)
    plt.xlabel('Missing Percentage (%)')
    plt.ylabel('Column')
    plt.tight_layout()
    plt.show()


## 2. Helper Functions for Lap Time Conversion

Qualifying times are stored as text strings (e.g., "1:16.285"). We need to convert to numeric for imputation, then convert back.


In [41]:
def lap_time_to_seconds(lap_time):
    """
    Convert lap time string to total seconds.
    
    Handles formats:
    - "1:16.285" (1 minute 16.285 seconds) → 76.285
    - "0:45.123" (45.123 seconds) → 45.123
    
    Returns NaN for missing values (\\N, empty strings, etc.)
    """
    if pd.isna(lap_time) or lap_time == '\\N' or lap_time == '' or str(lap_time).strip() == 'nan':
        return np.nan
    
    try:
        lap_time_str = str(lap_time).strip()
        
        # Handle format: M:SS.mmm or MM:SS.mmm
        if ':' in lap_time_str:
            parts = lap_time_str.split(':')
            if len(parts) == 2:
                minutes = float(parts[0])
                seconds = float(parts[1])
                return minutes * 60 + seconds
            elif len(parts) == 3:
                # Format: H:MM:SS.mmm (unlikely but handle it)
                hours = float(parts[0])
                minutes = float(parts[1])
                seconds = float(parts[2])
                return hours * 3600 + minutes * 60 + seconds
            else:
                # Unexpected format
                return np.nan
        else:
            # No colon, assume it's already in seconds format
            return float(lap_time_str)
    except (ValueError, AttributeError, TypeError):
        return np.nan


def seconds_to_lap_time(seconds):
    """
    Convert total seconds back to lap time string format "M:SS.mmm".
    
    Examples:
    - 76.285 → "1:16.285"
    - 45.123 → "0:45.123"
    - 0.5 → "0:00.500"
    """
    if pd.isna(seconds):
        return np.nan
    
    try:
        seconds = float(seconds)
        minutes = int(seconds // 60)
        secs = seconds % 60
        return f"{minutes}:{secs:06.3f}"  # Format: M:SS.mmm
    except (ValueError, TypeError):
        return np.nan


# Test the conversion functions
print("Testing lap time conversion functions:")
test_times = ["1:16.285", "0:45.123", "1:26.572", r"\N", np.nan, ""]
for test_time in test_times:
    seconds = lap_time_to_seconds(test_time)
    converted_back = seconds_to_lap_time(seconds)
    print(f"  '{test_time}' → {seconds} seconds → '{converted_back}'")

print("\n✓ Conversion functions ready")


Testing lap time conversion functions:
  '1:16.285' → 76.285 seconds → '1:16.285'
  '0:45.123' → 45.123 seconds → '0:45.123'
  '1:26.572' → 86.572 seconds → '1:26.572'
  '\N' → nan seconds → 'nan'
  'nan' → nan seconds → 'nan'
  '' → nan seconds → 'nan'

✓ Conversion functions ready


# 3. Remove placeholders

In [42]:
# Remove placeholder features (100% missing - FastF1 data not extracted)
placeholder_cols = [col for col in master.columns if any(x in col.lower() for x in 
    ['lap_time_variance', 'throttle_variance', 'overtake_attempts', 'avg_pit_stops'])]

print("Removing placeholder features (100% missing):")
removed_count = 0
for col in placeholder_cols:
    if col in master.columns:
        print(f"  - {col}")
        master = master.drop(columns=[col])
        removed_count += 1

print(f"\nRemoved {removed_count} placeholder features. New shape: {master.shape}")


Removing placeholder features (100% missing):
  - lap_time_variance
  - throttle_variance
  - overtake_attempts
  - avg_pit_stops

Removed 4 placeholder features. New shape: (12358, 96)


## 4. Handle Sprint Features

**Rationale:** Sprint races were introduced in 2021, so ~97% of historical races have missing sprint data. This is structural missingness.

**Method:** Impute with 0 (no sprint race = 0 points historically).


In [43]:
# Handle sprint features
sprint_cols = [col for col in master.columns if 'sprint' in col.lower()]

print("Handling sprint features (structural missingness - sprint races only 2021+):")
print("Method: Impute with 0 (no sprint race = 0 points)")

for col in sprint_cols:
    if col in master.columns:
        missing_before = master[col].isnull().sum()
        # Impute with 0 (no sprint race = 0 points/performance)
        master[col] = master[col].fillna(0)
        missing_after = master[col].isnull().sum()
        if missing_before > 0:
            print(f"  - {col}: {missing_before:,} missing → {missing_after:,} missing (imputed with 0)")

print(f"\n✓ Sprint features handled. All sprint features now have 0 missing values.")


Handling sprint features (structural missingness - sprint races only 2021+):
Method: Impute with 0 (no sprint race = 0 points)
  - sprint_results_resultId: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_constructorId: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_number: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_grid: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_position: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_positionText: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_positionOrder: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_points: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_laps: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_time: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_milliseconds: 11,998 missing → 0 missing (imputed with 0)
  - sprint_results_fastestLap: 11,998 missing → 0 missing (imputed w

## 2. Remove Unnecessary Columns Early

Remove columns that will be dropped anyway, so we don't waste time imputing them.


In [44]:

# Remove unnecessary columns early (before imputation)
print("="*70)
print("REMOVING UNNECESSARY COLUMNS EARLY")
print("="*70)
print("Removing columns that will be dropped anyway to avoid wasting time imputing them...")

columns_to_remove = [
    # Qualifying metadata
    'qualifyId',
    'qualifying_constructorId',
    'qualifying_number',
    'qualifying_position',
    
    # PositionText columns
    'positionText',
    'driver_standings_positionText',
    'constructor_standings_positionText',
    'sprint_results_positionText',
    
    # Wins
    'wins',
    
    # Circuit metadata (keeping lat/lng, removing text)
    'circuitRef',
    'circuit_name',
    'location',
    'country',
    'alt',
    'url',
    
    # Driver metadata
    'driverRef',
    'number_driver',
    # 'code',
    'forename',
    'surname',
    'dob',
    'nationality',
    'url_driver',
    
    # Constructor metadata
    'constructorRef',
    'name_constructor',
    'nationality_constructor',
    'url_constructor',
    'constructorResultsId',
    # In the columns_to_remove list, add:
    'constructor_results_points',  # Redundant - we use constructor_standings_points (cumulative) and _PRE_RACE versions
    
    # Sprint metadata
    'sprint_results_resultId',
    'sprint_results_constructorId',
    'sprint_results_number',
    'sprint_results_position',  # Keep positionOrder instead
    
    # Other
    'number',
    'status',  # We have statusId and status_category
    # 'name',
    
    # Should have been removed in 03.7
    'driver_avg_position_last_3',
]

# Only remove columns that exist
columns_removed = []
for col in columns_to_remove:
    if col in master.columns:
        master = master.drop(columns=[col])
        columns_removed.append(col)

print(f"  ✓ Removed {len(columns_removed)} columns")
if len(columns_removed) > 0:
    print(f"  Columns removed: {', '.join(columns_removed[:10])}{'...' if len(columns_removed) > 10 else ''}")
print(f"  New shape: {master.shape}")

print("\n" + "="*70)
print("✓ Unnecessary columns removed early")
print("="*70)


REMOVING UNNECESSARY COLUMNS EARLY
Removing columns that will be dropped anyway to avoid wasting time imputing them...


  ✓ Removed 35 columns
  Columns removed: qualifyId, qualifying_constructorId, qualifying_number, qualifying_position, positionText, driver_standings_positionText, constructor_standings_positionText, sprint_results_positionText, wins, circuitRef...
  New shape: (12358, 61)

✓ Unnecessary columns removed early


## 3. Identify Rows with Missing Rolling Features (First Races)

These rolling features require historical data and cannot be calculated for a driver's first race(s).
We need to identify and examine these rows before removing them.


In [45]:
# ========================================================================
# Identify and Display Rows with Missing Rolling Features
# ========================================================================
print("="*70)
print("IDENTIFYING ROWS WITH MISSING ROLLING FEATURES")
print("="*70)

# Features that require historical data (cannot be calculated for first races)
features_requiring_history = [
    'driver_points_avg_last_10',
    'driver_podium_rate_last_10',
    'finished_lapped_rate_last_10',
    'dnf_rate_last_10',
    'disqualified_rate_last_10',
    'not_classified_rate_last_10',
    'driver_avg_position_last_5',
    'driver_avg_grid_last_5'
]

# Find rows with any missing values in these features
rows_with_missing = master[features_requiring_history].isnull().any(axis=1)
rows_to_remove_count = rows_with_missing.sum()

if rows_to_remove_count > 0:
    print(f"\nFound {rows_to_remove_count:,} rows with missing values in rolling features")
    
    # Show all rows with missing values
    missing_rows_df = master[rows_with_missing].copy()
    
    # Select relevant columns for display
    display_cols = ['year', 'round', 'date', 'driverId', 'grid', 'positionOrder', 'points']
    display_cols.extend([col for col in features_requiring_history if col in missing_rows_df.columns])
    
    # Only show columns that exist
    display_cols = [col for col in display_cols if col in missing_rows_df.columns]
    
    print(f"\nAll {rows_to_remove_count:,} rows with missing rolling features:")
    print("="*70)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(missing_rows_df[display_cols].to_string())
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    print("="*70)
    
    # Show summary statistics
    print(f"\nSummary of rows to be removed:")
    print(f"  Date range: {missing_rows_df['date'].min()} to {missing_rows_df['date'].max()}")
    print(f"  Year range: {missing_rows_df['year'].min()} to {missing_rows_df['year'].max()}")
    print(f"  Unique drivers: {missing_rows_df['driverId'].nunique()}")
    if 'raceId' in missing_rows_df.columns:
        print(f"  Unique races: {missing_rows_df['raceId'].nunique()}")
    
    # Show which features are missing in each row
    print(f"\nMissing feature breakdown:")
    for col in features_requiring_history:
        if col in master.columns:
            missing_count = missing_rows_df[col].isnull().sum()
            if missing_count > 0:
                print(f"  - {col}: {missing_count:,} missing")
    
    # Verify these are early races (first races for drivers)
    print(f"\nVerifying these are first races:")
    if 'date' in missing_rows_df.columns:
        # Sort by date to see if they're at the beginning
        missing_rows_sorted = missing_rows_df.sort_values('date')
        print(f"  Earliest date: {missing_rows_sorted['date'].iloc[0]}")
        print(f"  Latest date: {missing_rows_sorted['date'].iloc[-1]}")
        print(f"  First 5 rows by date:")
        print(missing_rows_sorted[['year', 'date', 'driverId', 'grid']].head().to_string())
        
        # Check if these are indeed first races for each driver
        print(f"\n  Checking if these are first races for each driver:")
        for driver_id in missing_rows_df['driverId'].unique():
            driver_races = master[master['driverId'] == driver_id].sort_values('date')
            driver_missing = missing_rows_df[missing_rows_df['driverId'] == driver_id]
            if len(driver_races) > 0:
                first_race_date = driver_races['date'].iloc[0]
                missing_dates = driver_missing['date'].unique()
                is_first = all(d in [first_race_date] for d in missing_dates)
                print(f"    Driver {driver_id}: First race = {first_race_date}, Missing rows = {missing_dates}, Is first race? {is_first}")
    
else:
    print(f"\n✓ No rows found with missing values in rolling features")


IDENTIFYING ROWS WITH MISSING ROLLING FEATURES

Found 28 rows with missing values in rolling features

All 28 rows with missing rolling features:
    year  round       date  driverId  grid  points  driver_points_avg_last_10  driver_podium_rate_last_10  finished_lapped_rate_last_10  dnf_rate_last_10  disqualified_rate_last_10  not_classified_rate_last_10  driver_avg_position_last_5  driver_avg_grid_last_5
0   1994      1 1994-03-27        30     2    10.0                        NaN                         NaN                           NaN               NaN                        NaN                          NaN                         NaN                     NaN
1   1994      1 1994-03-27        71     4     6.0                        NaN                         NaN                           NaN               NaN                        NaN                          NaN                         NaN                     NaN
2   1994      1 1994-03-27        55     3     4.0                  

## 4. Rationale for Removing Rows with Missing Rolling Features

**Why these features cannot be imputed:**

These rolling features require historical data from previous races to calculate:
- `driver_points_avg_last_10`: Average points from last 10 races
- `driver_podium_rate_last_10`: Podium rate from last 10 races  
- `finished_lapped_rate_last_10`: Rate of finishing lapped from last 10 races
- `dnf_rate_last_10`: DNF rate from last 10 races
- `disqualified_rate_last_10`: Disqualification rate from last 10 races
- `not_classified_rate_last_10`: Not classified rate from last 10 races
- `driver_avg_position_last_5`: Average finishing position from last 5 races
- `driver_avg_grid_last_5`: Average grid position from last 5 races

**Why remove instead of impute:**

1. **No historical data exists**: These features are MEANINGLESS for a driver's first race(s) - there is no history to calculate from
2. **False information**: Imputing with 0 or adjacent grid positions would introduce false information that doesn't represent reality
3. **Data integrity**: These rows represent drivers' debut races where rolling statistics cannot legitimately exist
4. **Feature validity**: Removing ensures data integrity - we only keep rows where features can be legitimately calculated

**Impact:**
- Removes ~28 rows (first races for new drivers)
- Preserves data quality and feature meaning
- No information loss - these features would be invalid anyway
- Ensures all remaining rows have complete rolling feature data


In [46]:
# ========================================================================
# Remove Rows with Missing Rolling Features
# ========================================================================
print("="*70)
print("REMOVING ROWS WITH MISSING ROLLING FEATURES")
print("="*70)

# Features that require historical data (cannot be calculated for first races)
features_requiring_history = [
    'driver_points_avg_last_10',
    'driver_podium_rate_last_10',
    'finished_lapped_rate_last_10',
    'dnf_rate_last_10',
    'disqualified_rate_last_10',
    'not_classified_rate_last_10',
    'driver_avg_position_last_5',
    'driver_avg_grid_last_5'
]

# Find rows with any missing values in these features
rows_with_missing = master[features_requiring_history].isnull().any(axis=1)
rows_to_remove_count = rows_with_missing.sum()

if rows_to_remove_count > 0:
    rows_before = len(master)
    master = master[~rows_with_missing].reset_index(drop=True)
    rows_after = len(master)
    
    print(f"  Rows before removal: {rows_before:,}")
    print(f"  Rows removed: {rows_to_remove_count:,}")
    print(f"  Rows after removal: {rows_after:,}")
    
    # Verify all missing values are gone
    print(f"\n  Verifying removal:")
    all_removed = True
    for col in features_requiring_history:
        if col in master.columns:
            remaining_missing = master[col].isnull().sum()
            if remaining_missing > 0:
                print(f"    ⚠️ WARNING: {col} still has {remaining_missing:,} missing")
                all_removed = False
            else:
                print(f"    ✓ {col}: All missing values removed")
    
    if all_removed:
        print(f"\n  ✓ SUCCESS: All rows with missing rolling features removed")
        print(f"  ✓ Dataset now has {rows_after:,} rows (removed {rows_to_remove_count:,} first-race rows)")
    else:
        print(f"\n  ⚠️ WARNING: Some missing values remain - check imputation logic")
else:
    print(f"  ✓ No rows to remove")


REMOVING ROWS WITH MISSING ROLLING FEATURES
  Rows before removal: 12,358
  Rows removed: 28
  Rows after removal: 12,330

  Verifying removal:
    ✓ driver_points_avg_last_10: All missing values removed
    ✓ driver_podium_rate_last_10: All missing values removed
    ✓ finished_lapped_rate_last_10: All missing values removed
    ✓ dnf_rate_last_10: All missing values removed
    ✓ disqualified_rate_last_10: All missing values removed
    ✓ not_classified_rate_last_10: All missing values removed
    ✓ driver_avg_position_last_5: All missing values removed
    ✓ driver_avg_grid_last_5: All missing values removed

  ✓ SUCCESS: All rows with missing rolling features removed
  ✓ Dataset now has 12,330 rows (removed 28 first-race rows)


## 5. Enhanced Missing Value Handling (Non-Qualifying Features)

This section handles non-qualifying features that need sophisticated imputation:

1. **Circuit podium rates** (fill with 0 if no prior races at circuit)
2. **positionOrder_trend_season handling** (first race = 0, then forward-fill within driver-season)

**Note:** 
- Constructor standings are CALCULATED (not imputed) in Cell 25 from driver standings
- Driver rolling features are handled in Cell 22 (excluding features removed in Cell 3)
- Qualifying features (q1, q2, q3) are handled separately in the next section with proper grid constraints.


In [47]:
# Enhanced missing value handling for non-qualifying features

print("="*70)
print("ENHANCED MISSING VALUE HANDLING (NON-QUALIFYING FEATURES)")
print("="*70)

# ========================================================================
# STEP 1: Fill Circuit Podium Rates with 0 (no prior races)
# ========================================================================
print("\n2. Filling circuit podium rates with 0 (no prior races at circuit)...")
for col in ['driver_podium_rate_at_circuit', 'constructor_podium_rate_at_circuit']:
    if col in master.columns:
        missing_before = master[col].isnull().sum()
        if missing_before > 0:
            # Fill with 0 if driver/constructor has no prior races at this circuit
            master[col] = master[col].fillna(0)
            missing_after = master[col].isnull().sum()
            print(f"  ✓ {col}: {missing_before:,} → {missing_after:,} missing (filled with 0)")

# ========================================================================
# STEP 2: Handle positionOrder_trend_season
# ========================================================================
print("\n4. Handling positionOrder_trend_season...")
if 'positionOrder_trend_season' in master.columns:
    missing_before = master['positionOrder_trend_season'].isnull().sum()
    if missing_before > 0:
        # OPTIMIZATION: Only sort if not already sorted, and reset index
        # Check if already sorted (quick check on first few rows)
        needs_sort = False
        if len(master) > 1:
            sample = master.head(100)
            if not (sample.groupby(['driverId', 'year']).apply(
                lambda g: g['date'].is_monotonic_increasing if 'date' in g.columns else True
            ).all() if len(sample) > 0 else True):
                needs_sort = True
        
        if needs_sort:
            master = master.sort_values(['driverId', 'year', 'round', 'date']).reset_index(drop=True)
        else:
            master = master.reset_index(drop=True)
        
        # Fill first race of each driver-season with 0 (more efficient)
        grouped = master.groupby(['driverId', 'year'])
        first_race_mask = grouped.cumcount() == 0
        first_race_filled = (first_race_mask & master['positionOrder_trend_season'].isnull()).sum()
        master.loc[first_race_mask & master['positionOrder_trend_season'].isnull(), 'positionOrder_trend_season'] = 0
        
        # Forward-fill within driver-season groups (more efficient)
        after_first = master['positionOrder_trend_season'].isnull().sum()
        master['positionOrder_trend_season'] = grouped['positionOrder_trend_season'].ffill()
        after_ffill = master['positionOrder_trend_season'].isnull().sum()
        ffill_filled = after_first - after_ffill
        
        # Fill any remaining with 0
        master['positionOrder_trend_season'] = master['positionOrder_trend_season'].fillna(0)
        
        missing_after = master['positionOrder_trend_season'].isnull().sum()
        print(f"  ✓ positionOrder_trend_season: {missing_before:,} → {missing_after:,} missing")
        print(f"    - First race filled with 0: {first_race_filled:,}")
        print(f"    - Forward-fill within driver-season: {ffill_filled:,}")
        print(f"    - Remaining filled with 0: {missing_before - first_race_filled - ffill_filled - missing_after:,}")


ENHANCED MISSING VALUE HANDLING (NON-QUALIFYING FEATURES)

2. Filling circuit podium rates with 0 (no prior races at circuit)...
  ✓ driver_podium_rate_at_circuit: 3,182 → 0 missing (filled with 0)
  ✓ constructor_podium_rate_at_circuit: 1,126 → 0 missing (filled with 0)

4. Handling positionOrder_trend_season...
  ✓ positionOrder_trend_season: 3,644 → 0 missing
    - First race filled with 0: 778
    - Forward-fill within driver-season: 2,866
    - Remaining filled with 0: 0


# Qualifying Imputation


In [48]:
def impute_qualifying_times_track_specific(df, col, race_col='raceId', grid_col='grid', 
                                           date_col='date', circuit_col='circuitId'):
    """
    Impute qualifying times using track-specific strategy.
    PRIORITIZES CORRECTNESS OVER EFFICIENCY - uses simple, robust methods.
    Returns (df, method_counts) where method_counts tracks which method was used.
    """
    import pandas as pd
    import numpy as np

    # Initialize method_counts to track which imputation method was used
    method_counts = {
        'same_race_adjacent': 0,
        'previous_track_race': 0,
        'same_season_median': 0,
        'global_median': 0
    }
    
    # Defensive checks for known crash sources:
    # 1. Needed columns present
    required_cols = [col, grid_col, race_col, date_col, circuit_col, 'year']
    for req in required_cols:
        if req not in df.columns:
            raise ValueError(f"Missing required column: {req}")
    # 2. Functions needed are defined
    if 'lap_time_to_seconds' not in globals() or 'seconds_to_lap_time' not in globals():
        raise RuntimeError("lap_time_to_seconds or seconds_to_lap_time not defined in globals")
    
    # CRITICAL: Remove rows with null merge keys FIRST - merge_asof cannot handle nulls
    # This must be done at the very start to prevent any merge_asof errors
    null_mask = df[[circuit_col, date_col]].isnull().any(axis=1)
    if null_mask.sum() > 0:
        print(f"  [Warning] Dropping {null_mask.sum():,} rows with null values in {circuit_col} or {date_col}")
        df = df[~null_mask].copy()
    
    # CRITICAL: Sort by circuit_col and date_col at the start - required for merge_asof
    df = df.sort_values([circuit_col, date_col], ascending=[True, True]).reset_index(drop=True)

    # Step 1: Convert text times to numeric seconds (vectorized)
    try:
        df[f'{col}_seconds'] = df[col].apply(lap_time_to_seconds)
    except Exception as e:
        print(f"Error converting '{col}' to seconds: {e}")
        raise

    # Step 2: Determine which drivers should have this session time
    try:
        # grid_col may be float; ensure int for comparison!
        grid_series = df[grid_col]
        if pd.api.types.is_float_dtype(grid_series):
            # If NaN, conversion will work as intended (np.nan stays nan)
            grid_series = grid_series.round().astype('Int64')
        if col == 'q1':
            should_have_time = grid_series.notna()
        elif col == 'q2':
            should_have_time = (grid_series <= 15) & grid_series.notna()
        elif col == 'q3':
            should_have_time = (grid_series <= 10) & grid_series.notna()
        else:
            should_have_time = grid_series.notna()
    except Exception as e:
        print(f"Error building should_have_time mask for {col}: {e}")
        raise

    missing_mask = should_have_time & df[f'{col}_seconds'].isnull()

    if missing_mask.sum() == 0:
        df[col] = df[f'{col}_seconds'].apply(seconds_to_lap_time)
        df = df.drop(columns=[f'{col}_seconds'])
        return df

    # Ensure grid_col is numeric for lookup/indexing
    if not pd.api.types.is_numeric_dtype(df[grid_col]):
        try:
            df[grid_col] = pd.to_numeric(df[grid_col], errors='coerce')
        except Exception as e:
            print(f"Could not convert column {grid_col} to numeric: {e}")
            raise

    # Step 3: Same-race, adjacent grid positions
    try:
        for race_id in df[race_col].unique():
            race_mask = df[race_col] == race_id
            race_data = df[race_mask].sort_values(grid_col).copy()
            missing_in_race_mask = race_mask & missing_mask

            if missing_in_race_mask.sum() == 0:
                continue

            # Remove NaN grids
            race_data = race_data[race_data[grid_col].notna()]
            # Create grid position mapping for this race
            # Use int version of grid_col for mapping
            grid_keys = race_data[grid_col].astype(int)
            race_grid_map = pd.Series(
                race_data[f'{col}_seconds'].values, index=grid_keys
            ).to_dict()

            for idx in df[missing_in_race_mask].index:
                grid_pos = df.loc[idx, grid_col]
                if np.isnan(grid_pos):
                    continue
                grid_pos = int(grid_pos)
                candidates = []
                for offset in range(1, 6):
                    for direction in [-1, 1]:
                        adj_grid = grid_pos + (offset * direction)
                        if adj_grid in race_grid_map and pd.notna(race_grid_map[adj_grid]):
                            candidates.append(race_grid_map[adj_grid])
                        if len(candidates) >= 2:
                            break
                    if len(candidates) >= 2:
                        break
                if len(candidates) > 0:
                    df.loc[idx, f'{col}_seconds'] = np.median(candidates)
                method_counts['same_race_adjacent'] += 1
    except Exception as e:
        print("Error during same-race adjacent grid imputation:", e)
        raise

    # Step 4: Previous race at same track imputation
    # SIMPLE APPROACH: Loop through missing values and find previous race manually
    # This avoids merge_asof complexity and ensures correctness
    still_missing = should_have_time & df[f'{col}_seconds'].isnull()
    
    if still_missing.sum() > 0:
        # Get all races with valid times, sorted by circuit and date
        valid_races = df[df[f'{col}_seconds'].notna()].copy()
        valid_races = valid_races.sort_values([circuit_col, date_col], ascending=[True, True]).reset_index(drop=True)
        
        # For each missing value, find previous race at same circuit
        missing_indices = df[still_missing].index.tolist()
        
        for idx in missing_indices:
            circuit_id = df.loc[idx, circuit_col]
            race_date = df.loc[idx, date_col]
            grid_pos = df.loc[idx, grid_col]
            
            # Find previous races at same circuit with valid times
            prev_races = valid_races[
                (valid_races[circuit_col] == circuit_id) & 
                (valid_races[date_col] < race_date)
            ]
            
            if len(prev_races) > 0:
                # Get the most recent previous race
                prev_race = prev_races.iloc[-1]
                prev_race_id = prev_race[race_col]
                
                # Get all times from that previous race
                prev_race_times = df[
                    (df[race_col] == prev_race_id) & 
                    (df[f'{col}_seconds'].notna())
                ][f'{col}_seconds'].values
                
                if len(prev_race_times) > 0:
                    min_time = prev_race_times.min()
                    max_time = prev_race_times.max()
                    max_grid = df[df[race_col] == prev_race_id][grid_col].max()
                    
                    # Impute based on grid position
                    if pd.notna(grid_pos):
                        grid_pos = int(grid_pos)
                        if grid_pos == 1:
                            df.loc[idx, f'{col}_seconds'] = min_time
                            method_counts['previous_track_race'] += 1
                        elif pd.notna(max_grid) and grid_pos >= max_grid:
                            df.loc[idx, f'{col}_seconds'] = max_time
                            method_counts['previous_track_race'] += 1
                        elif pd.notna(max_grid) and max_grid > 1:
                            progress = (grid_pos - 1) / (max_grid - 1)
                            df.loc[idx, f'{col}_seconds'] = min_time + progress * (max_time - min_time)
                            method_counts['previous_track_race'] += 1

    # Step 5: Fallback to same-season median
    still_missing_after_prev = should_have_time & df[f'{col}_seconds'].isnull()
    
    if still_missing_after_prev.sum() > 0:
        # Compute same-season medians
        same_season_medians = df.groupby(['year', grid_col])[f'{col}_seconds'].median().reset_index()
        same_season_medians.columns = ['year', grid_col, f'{col}_same_season_median']
        
        # Loop through missing values and impute
        missing_indices = df[still_missing_after_prev].index.tolist()
        for idx in missing_indices:
            year = df.loc[idx, 'year']
            grid_pos = df.loc[idx, grid_col]
            
            if pd.notna(year) and pd.notna(grid_pos):
                matching = same_season_medians[
                    (same_season_medians['year'] == year) & 
                    (same_season_medians[grid_col] == grid_pos)
                ]
                if len(matching) > 0 and pd.notna(matching.iloc[0][f'{col}_same_season_median']):
                    df.loc[idx, f'{col}_seconds'] = matching.iloc[0][f'{col}_same_season_median']
                    method_counts['same_season_median'] += 1

    # Step 6: Fallback to global median
    still_missing_after_season = should_have_time & df[f'{col}_seconds'].isnull()
    
    if still_missing_after_season.sum() > 0:
        # Compute global medians
        global_medians = df.groupby(grid_col)[f'{col}_seconds'].median().reset_index()
        global_medians.columns = [grid_col, f'{col}_global_median']
        
        # Loop through missing values and impute
        missing_indices = df[still_missing_after_season].index.tolist()
        for idx in missing_indices:
            grid_pos = df.loc[idx, grid_col]
            
            if pd.notna(grid_pos):
                matching = global_medians[global_medians[grid_col] == grid_pos]
                if len(matching) > 0 and pd.notna(matching.iloc[0][f'{col}_global_median']):
                    df.loc[idx, f'{col}_seconds'] = matching.iloc[0][f'{col}_global_median']
                    method_counts['global_median'] += 1

    # Step 7: Convert back to text format
    df[col] = df[f'{col}_seconds'].apply(seconds_to_lap_time)
    df = df.drop(columns=[f'{col}_seconds'])
    
    return df, method_counts

print("✓ Track-specific qualifying imputation function loaded (robust version - prioritizes correctness)")

✓ Track-specific qualifying imputation function loaded (robust version - prioritizes correctness)


In [49]:
# Handle qualifying features with proper grid constraints

import numpy as np
import pandas as pd

# Simple, clear check: raise error if "master" is not defined
if 'master' not in globals():
    raise NameError("The dataframe 'master' is not defined in this notebook. Please ensure it is loaded (e.g., via pd.read_csv or previous cell) before running this cell.")

print("="*70)
print("HANDLING QUALIFYING FEATURES")
print("="*70)

# STEP 1: Remove invalid Q2/Q3 times for drivers who shouldn't have them
print("\nSTEP 1: Removing invalid Q2/Q3 times...")

if 'grid' not in master.columns or 'q2' not in master.columns or 'q3' not in master.columns:
    raise KeyError("Required columns ('grid', 'q2', 'q3') not found in master dataframe.")

# Remove Q2 for grid > 15 (should only have Q1)
q2_should_remove = (master['grid'] > 15) & master['q2'].notna()
q2_removed_count = q2_should_remove.sum()
if q2_removed_count > 0:
    print(f"  Removing Q2 for {q2_removed_count:,} rows with grid > 15")
    master.loc[q2_should_remove, 'q2'] = np.nan

# Remove Q3 for grid > 10 (should only have Q1, Q2)
q3_should_remove = (master['grid'] > 10) & master['q3'].notna()
q3_removed_count = q3_should_remove.sum()
if q3_removed_count > 0:
    print(f"  Removing Q3 for {q3_removed_count:,} rows with grid > 10")
    master.loc[q3_should_remove, 'q3'] = np.nan

if q2_removed_count > 0 or q3_removed_count > 0:
    print(f"  ✓ Cleaned up {q2_removed_count + q3_removed_count:,} invalid Q2/Q3 times")
else:
    print(f"  ✓ No invalid Q2/Q3 times found")

# STEP 2: Apply track-specific imputation for qualifying times
print("\nSTEP 2: Applying track-specific imputation for qualifying times...")
qualifying_time_cols = ['q1', 'q2', 'q3']

for col in qualifying_time_cols:
    if col in master.columns:
        missing_before = master[col].isnull().sum()
        
        if missing_before > 0:
            print(f"\n  Imputing {col} using track-specific method...")
            
            # CRITICAL: Ensure proper sorting and no nulls in merge keys BEFORE calling function
            # The function handles this internally, but doing it here ensures consistency
            if 'date' in master.columns and 'circuitId' in master.columns:
                # Remove rows with null merge keys (they cannot be imputed using track-specific method)
                null_mask = master[['circuitId', 'date']].isnull().any(axis=1)
                if null_mask.sum() > 0:
                    print(f"  [Warning] {null_mask.sum():,} rows have null values in circuitId or date - these will be excluded from track-specific imputation")
                    original_shape = master.shape[0]
                    master = master[~null_mask].copy()
                    print(f"  [Info] Excluded {original_shape - master.shape[0]:,} rows with null merge keys")
                
                # Sort by circuitId and date (ascending) - REQUIRED for proper imputation
                master = master.sort_values(['circuitId', 'date'], ascending=[True, True]).reset_index(drop=True)
            
            # Call imputation function - simple call, no retry loops needed
            # The function is now robust and handles all edge cases internally
            try:
                master, method_counts = impute_qualifying_times_track_specific(
                    master, col, race_col='raceId', grid_col='grid', 
                    date_col='date', circuit_col='circuitId'
                )
            except Exception as e:
                print(f"  [Error] Failed to impute {col}: {e}")
                raise

            missing_after = master[col].isnull().sum()
            filled_total = missing_before - missing_after

            # Show which method succeeded
            print(f"    ✓ {col}: {missing_before:,} missing → {missing_after:,} missing")
            if filled_total > 0:
                print(f"      Methods used:")
                if method_counts.get('same_race_adjacent', 0) > 0:
                    print(f"        - Same-race adjacent grid: {method_counts.get('same_race_adjacent', 0):,} filled")
                if method_counts.get('previous_track_race', 0) > 0:
                    print(f"        - Previous race at same track: {method_counts.get('previous_track_race', 0):,} filled")
                if method_counts.get('same_season_median', 0) > 0:
                    print(f"        - Same-season median: {method_counts.get('same_season_median', 0):,} filled")
                if method_counts.get('global_median', 0) > 0:
                    print(f"        - Global median: {method_counts.get('global_median', 0):,} filled")
            
            # Verify grid constraints are respected
            if col == 'q2':
                should_have = (master['grid'] <= 15) & master['grid'].notna()
                still_missing_q2 = (should_have & master[col].isnull()).sum()
                invalid_q2 = ((master['grid'] > 15) & master[col].notna()).sum()
                print(f"      - Should have time (grid <= 15): {still_missing_q2:,} still missing")
                print(f"      - Legitimately missing (grid > 15): {(~should_have & master[col].isnull()).sum():,}")
                if invalid_q2 > 0:
                    print(f"      ⚠️ WARNING: {invalid_q2:,} rows with grid > 15 still have Q2!")
                else:
                    print(f"      ✓ Verified: No Q2 for grid > 15")
            elif col == 'q3':
                should_have = (master['grid'] <= 10) & master['grid'].notna()
                still_missing_q3 = (should_have & master[col].isnull()).sum()
                invalid_q3 = ((master['grid'] > 10) & master[col].notna()).sum()
                print(f"      - Should have time (grid <= 10): {still_missing_q3:,} still missing")
                print(f"      - Legitimately missing (grid > 10): {(~should_have & master[col].isnull()).sum():,}")
                if invalid_q3 > 0:
                    print(f"      ⚠️ WARNING: {invalid_q3:,} rows with grid > 10 still have Q3!")
                else:
                    print(f"      ✓ Verified: No Q3 for grid > 10")

# STEP 3: Handle other qualifying columns (non-time columns)
print("\nSTEP 3: Handling other qualifying columns (non-time)...")
qualifying_cols = [col for col in master.columns if any(x in col.lower() for x in 
    ['qualify', 'q1', 'q2', 'q3']) and col not in qualifying_time_cols]

for col in qualifying_cols:
    if col in master.columns:
        missing_before = master[col].isnull().sum()
        
        if missing_before > 0:
            # Use adjacent grid positions in same race for numeric columns
            if master[col].dtype in [np.int64, np.float64, float, int]:
                filled_count = 0
                for race_id in master['raceId'].unique():
                    race_mask = master['raceId'] == race_id
                    # Always sort by grid for stable 'adjacent' logic
                    race_data = master[race_mask].sort_values('grid').copy()
                    
                    missing_mask = race_mask & master[col].isnull()
                    missing_idx = race_data[missing_mask].index
                    
                    for idx in missing_idx:
                        grid_pos = race_data.loc[idx, 'grid']
                        
                        # Get adjacent grid positions
                        candidates = []
                        for offset in range(1, 6):
                            for direction in [-1, 1]:
                                adj_grid = grid_pos + (offset * direction)
                                adj_values = race_data[
                                    (race_data['grid'] == adj_grid) & 
                                    (race_data[col].notna())
                                ][col].values
                                candidates.extend(adj_values)
                        
                        if len(candidates) > 0:
                            master.loc[idx, col] = np.median(candidates)
                            filled_count += 1
                
                missing_after = master[col].isnull().sum()
                if missing_before > 0:
                    method_str = f"adjacent grid positions ({filled_count:,} filled)" if filled_count > 0 else "no change"
                    print(f"  - {col}: {missing_before:,} missing → {missing_after:,} missing ({method_str})")

print("\n" + "="*70)
print("✓ Qualifying features handled!")
print("="*70)


HANDLING QUALIFYING FEATURES

STEP 1: Removing invalid Q2/Q3 times...
  Removing Q2 for 2,983 rows with grid > 15
  Removing Q3 for 5,415 rows with grid > 10
  ✓ Cleaned up 8,398 invalid Q2/Q3 times

STEP 2: Applying track-specific imputation for qualifying times...

  Imputing q1 using track-specific method...
    ✓ q1: 1,862 missing → 0 missing
      Methods used:
        - Same-race adjacent grid: 2,018 filled
        - Previous race at same track: 1,693 filled
        - Same-season median: 136 filled

  Imputing q2 using track-specific method...
    ✓ q2: 4,867 missing → 3,572 missing
      Methods used:
        - Same-race adjacent grid: 3,215 filled
        - Same-season median: 195 filled
        - Global median: 2,775 filled
      - Should have time (grid <= 15): 0 still missing
      - Legitimately missing (grid > 15): 3,572
      ✓ Verified: No Q2 for grid > 15

  Imputing q3 using track-specific method...
    ✓ q3: 7,323 missing → 6,445 missing
      Methods used:
        - 

## 9. Calculate Driver Standings Features

**Rationale:** Driver standings are CALCULATED from race points, not imputed. Constructor standings depend on driver standings.

**Method:** 
- `driver_standings_points` = Cumulative sum of `points` for each driver
- `driver_standings_position` = Rank of drivers by cumulative points at each race
- `wins` = Count of races where `positionOrder == 1` (calculated, not imputed)


In [50]:
# Helper function for imputing by adjacent constructor positions
def impute_by_adjacent_constructor_position(df, col, race_col='raceId', 
                                            constructor_pos_col='constructor_standings_position',
                                            max_offset=5):
    """
    Impute constructor features using adjacent constructor standings positions in same race.
    Returns (df, missing_before, missing_after, filled_count)
    """
    if col not in df.columns or df[col].dtype not in [np.int64, np.float64]:
        return df, 0, 0, 0
    
    missing_before = df[col].isnull().sum()
    if missing_before == 0:
        return df, 0, 0, 0
    
    filled_count = 0
    # Impute using adjacent constructor positions in same race
    for race_id in df[race_col].unique():
        race_mask = df[race_col] == race_id
        race_constructors = df[race_mask].copy()
        
        if constructor_pos_col in df.columns:
            # Create mapping of constructorId to standings position
            constructor_pos_map = race_constructors.groupby('constructorId')[constructor_pos_col].first()
            
            missing_mask = race_mask & df[col].isnull()
            missing_idx = race_constructors[missing_mask].index
            
            for idx in missing_idx:
                constructor_id = race_constructors.loc[idx, 'constructorId']
                if constructor_id in constructor_pos_map.index:
                    constructor_pos = constructor_pos_map[constructor_id]
                    
                    # Find constructors with adjacent positions
                    candidates = []
                    for offset in range(1, max_offset + 1):
                        for direction in [-1, 1]:
                            adj_pos = constructor_pos + (offset * direction)
                            adj_constructors = constructor_pos_map[constructor_pos_map == adj_pos].index
                            
                            for adj_constructor in adj_constructors:
                                adj_values = race_constructors[
                                    (race_constructors['constructorId'] == adj_constructor) &
                                    (race_constructors[col].notna())
                                ][col].values
                                candidates.extend(adj_values)
                            
                            if len(candidates) >= 2:
                                break
                        if len(candidates) >= 2:
                            break
                    
                    if len(candidates) > 0:
                        df.loc[idx, col] = np.median(candidates)
                        filled_count += 1
    
    missing_after = df[col].isnull().sum()
    return df, missing_before, missing_after, filled_count


# ========================================================================
# Calculate Driver Standings (CALCULATED, not imputed) - MUST BE FIRST
# ========================================================================
print("="*70)
print("CALCULATING DRIVER STANDINGS")
print("="*70)
print("Method: Calculate from cumulative race points")

# Sort by year, round, date for proper temporal ordering
master = master.sort_values(['year', 'round', 'date']).reset_index(drop=True)

# Calculate driver_standings_points: cumulative sum of points per driver
print("\nCalculating driver_standings_points...")
missing_before_points = master['driver_standings_points'].isnull().sum() if 'driver_standings_points' in master.columns else 0

if 'points' in master.columns:
    # Calculate cumulative points per driver
    master['driver_standings_points'] = master.groupby('driverId')['points'].cumsum()
    missing_after_points = master['driver_standings_points'].isnull().sum() if 'driver_standings_points' in master.columns else 0
    if missing_before_points > 0:
        print(f"  ✓ driver_standings_points: {missing_before_points:,} → {missing_after_points:,} missing (calculated from cumulative points)")
else:
    print("  ⚠️ WARNING: 'points' column not found - cannot calculate driver_standings_points")

# Calculate driver_standings_position: rank by cumulative points at each race
print("\nCalculating driver_standings_position...")
missing_before_pos = master['driver_standings_position'].isnull().sum() if 'driver_standings_position' in master.columns else 0

if 'driver_standings_points' in master.columns:
    for race_id in master['raceId'].unique():
        race_mask = master['raceId'] == race_id
        race_data = master[race_mask].copy()
        
        # Sort by cumulative points (descending), then by driverId for ties
        race_data = race_data.sort_values(['driver_standings_points', 'driverId'], ascending=[False, True])
        race_data['driver_standings_position'] = range(1, len(race_data) + 1)
        master.loc[race_mask, 'driver_standings_position'] = race_data['driver_standings_position'].values
    
    missing_after_pos = master['driver_standings_position'].isnull().sum() if 'driver_standings_position' in master.columns else 0
    if missing_before_pos > 0:
        print(f"  ✓ driver_standings_position: {missing_before_pos:,} → {missing_after_pos:,} missing (calculated from ranking)")
else:
    print("  ⚠️ WARNING: 'driver_standings_points' not available - cannot calculate position")

# Calculate wins: count races where positionOrder == 1
print("\nCalculating wins...")
if 'wins' in master.columns and 'positionOrder' in master.columns:
    missing_before_wins = master['wins'].isnull().sum()
    
    # Count wins per driver up to each race (cumulative)
    master['is_win'] = (master['positionOrder'] == 1).astype(int)
    master['wins'] = master.groupby('driverId')['is_win'].cumsum()
    master = master.drop(columns=['is_win'])
    
    missing_after_wins = master['wins'].isnull().sum()
    if missing_before_wins > 0:
        print(f"  ✓ wins: {missing_before_wins:,} → {missing_after_wins:,} missing (calculated from positionOrder)")
else:
    if 'wins' in master.columns:
        print("  ⚠️ WARNING: 'positionOrder' column not found - cannot calculate wins")

print(f"\n✓ Driver standings features calculated.")

# Sort by year, round, date for proper temporal ordering
master = master.sort_values(['year', 'round', 'date']).reset_index(drop=True)

# Calculate constructor standings points: sum of driver standings points for all drivers in that constructor
print("\nCalculating constructor_standings_points...")
missing_before_points = master['constructor_standings_points'].isnull().sum() if 'constructor_standings_points' in master.columns else 0

for race_id in master['raceId'].unique():
    race_mask = master['raceId'] == race_id
    race_data = master[race_mask].copy()
    
    # Sum driver_standings_points by constructor
    if 'driver_standings_points' in master.columns:
        constructor_points = race_data.groupby('constructorId')['driver_standings_points'].sum()
        # Assign to each row
        master.loc[race_mask, 'constructor_standings_points'] = master.loc[race_mask, 'constructorId'].map(constructor_points)

missing_after_points = master['constructor_standings_points'].isnull().sum() if 'constructor_standings_points' in master.columns else 0
if missing_before_points > 0:
    print(f"  ✓ constructor_standings_points: {missing_before_points:,} → {missing_after_points:,} missing (calculated from driver standings)")

# Calculate constructor standings position: rank by total points
print("\nCalculating constructor_standings_position...")
missing_before_pos = master['constructor_standings_position'].isnull().sum() if 'constructor_standings_position' in master.columns else 0

for race_id in master['raceId'].unique():
    race_mask = master['raceId'] == race_id
    race_data = master[race_mask].copy()
    
    # Get unique constructors and their total points
    constructor_totals = race_data.groupby('constructorId')['constructor_standings_points'].first()
    # Rank by points (descending), then by constructorId for ties
    constructor_totals_sorted = constructor_totals.sort_values(ascending=False)
    constructor_rank = constructor_totals_sorted.rank(method='min', ascending=False).astype(int)
    
    # Assign rank to each row
    master.loc[race_mask, 'constructor_standings_position'] = master.loc[race_mask, 'constructorId'].map(constructor_rank)

missing_after_pos = master['constructor_standings_position'].isnull().sum() if 'constructor_standings_position' in master.columns else 0
if missing_before_pos > 0:
    print(f"  ✓ constructor_standings_position: {missing_before_pos:,} → {missing_after_pos:,} missing (calculated from constructor points)")

# Handle constructor rolling features (non-standings)
constructor_rolling = [col for col in rolling_cols if 'constructor_' in col and 'standings' not in col.lower()]

if len(constructor_rolling) > 0:
    print("\nHandling constructor rolling features (non-standings):")
    print("Method: Adjacent constructor standings positions → forward-fill fallback")
    
    for col in constructor_rolling:
        if col in master.columns and master[col].dtype in [np.int64, np.float64]:
            # Step 1: Try adjacent constructor positions
            master, missing_before, missing_after_step1, filled_adjacent = impute_by_adjacent_constructor_position(
                master, col, race_col='raceId',
                constructor_pos_col='constructor_standings_position',
                max_offset=5
            )
            
            # Step 2: Forward-fill within constructor groups (if still missing)
            still_missing = master[col].isnull()
            if still_missing.sum() > 0:
                missing_before_step2 = still_missing.sum()
                master[col] = master.groupby('constructorId')[col].ffill()
                missing_after_ffill = master[col].isnull().sum()
                filled_ffill = missing_before_step2 - missing_after_ffill
                
                # Step 3: Fill remaining with same-race median or 0
                still_missing = master[col].isnull()
                if still_missing.sum() > 0:
                    missing_before_step3 = still_missing.sum()
                    # Try same-race median first
                    for race_id in master[still_missing]['raceId'].unique():
                        race_mask = master['raceId'] == race_id
                        race_median = master[race_mask & master[col].notna()][col].median()
                        if pd.notna(race_median):
                            master.loc[race_mask & still_missing, col] = race_median
                    
                    # Fill any remaining with 0
                    master[col] = master[col].fillna(0)
                    missing_after = master[col].isnull().sum()
                    filled_step3 = missing_before_step3 - missing_after
                else:
                    missing_after = missing_after_ffill
                    filled_step3 = 0
            else:
                missing_after = missing_after_step1
                filled_ffill = 0
                filled_step3 = 0
            
            if missing_before > 0:
                methods_used = []
                if filled_adjacent > 0:
                    methods_used.append(f"adjacent constructor positions ({filled_adjacent:,} filled)")
                if filled_ffill > 0:
                    methods_used.append(f"forward-fill within constructor ({filled_ffill:,} filled)")
                if filled_step3 > 0:
                    methods_used.append(f"same-race median/0 ({filled_step3:,} filled)")
                method_str = " → ".join(methods_used) if methods_used else "no change"
                print(f"  ✓ {col}: {missing_before:,} → {missing_after:,} missing ({method_str})")

print(f"\n✓ Constructor standings calculated and rolling features handled.")


CALCULATING DRIVER STANDINGS
Method: Calculate from cumulative race points

Calculating driver_standings_points...
  ✓ driver_standings_points: 313 → 0 missing (calculated from cumulative points)

Calculating driver_standings_position...
  ✓ driver_standings_position: 313 → 0 missing (calculated from ranking)

Calculating wins...

✓ Driver standings features calculated.

Calculating constructor_standings_points...
  ✓ constructor_standings_points: 118 → 0 missing (calculated from driver standings)

Calculating constructor_standings_position...
  ✓ constructor_standings_position: 118 → 0 missing (calculated from constructor points)

Handling constructor rolling features (non-standings):
Method: Adjacent constructor standings positions → forward-fill fallback

✓ Constructor standings calculated and rolling features handled.


## 10. Calculate Constructor Standings Features

**Rationale:** Constructor standings are CALCULATED from driver standings points, not imputed.

**Method:** 
- `constructor_standings_points` = Sum of `driver_standings_points` for all drivers in that constructor at that race
- `constructor_standings_position` = Rank of constructors by total points at that race
- Constructor rolling features (non-standings) are still imputed using adjacent positions


In [51]:
# ========================================================================
# Calculate Constructor Standings (CALCULATED, not imputed)
# ========================================================================
print("="*70)
print("CALCULATING CONSTRUCTOR STANDINGS")
print("="*70)
print("Method: Calculate from driver standings points (sum by constructor)")

# Ensure driver standings are already calculated (from Cell 19)
if 'driver_standings_points' in master.columns and master['driver_standings_points'].isnull().sum() > 0:
    print("  ⚠️ WARNING: driver_standings_points still has missing values - constructor standings may be incomplete")

# Sort by year, round, date for proper temporal ordering
master = master.sort_values(['year', 'round', 'date']).reset_index(drop=True)

# Calculate constructor standings points: sum of driver standings points for all drivers in that constructor
print("\nCalculating constructor_standings_points...")
missing_before_points = master['constructor_standings_points'].isnull().sum() if 'constructor_standings_points' in master.columns else 0

for race_id in master['raceId'].unique():
    race_mask = master['raceId'] == race_id
    race_data = master[race_mask].copy()
    
    # Sum driver_standings_points by constructor
    if 'driver_standings_points' in master.columns:
        constructor_points = race_data.groupby('constructorId')['driver_standings_points'].sum()
        # Assign to each row
        master.loc[race_mask, 'constructor_standings_points'] = master.loc[race_mask, 'constructorId'].map(constructor_points)

missing_after_points = master['constructor_standings_points'].isnull().sum() if 'constructor_standings_points' in master.columns else 0
if missing_before_points > 0:
    print(f"  ✓ constructor_standings_points: {missing_before_points:,} → {missing_after_points:,} missing (calculated from driver standings)")

# Calculate constructor standings position: rank by total points
print("\nCalculating constructor_standings_position...")
missing_before_pos = master['constructor_standings_position'].isnull().sum() if 'constructor_standings_position' in master.columns else 0

for race_id in master['raceId'].unique():
    race_mask = master['raceId'] == race_id
    race_data = master[race_mask].copy()
    
    # Get unique constructors and their total points
    constructor_totals = race_data.groupby('constructorId')['constructor_standings_points'].first()
    # Rank by points (descending), then by constructorId for ties
    constructor_totals_sorted = constructor_totals.sort_values(ascending=False)
    constructor_rank = constructor_totals_sorted.rank(method='min', ascending=False).astype(int)
    
    # Assign rank to each row
    master.loc[race_mask, 'constructor_standings_position'] = master.loc[race_mask, 'constructorId'].map(constructor_rank)

missing_after_pos = master['constructor_standings_position'].isnull().sum() if 'constructor_standings_position' in master.columns else 0
if missing_before_pos > 0:
    print(f"  ✓ constructor_standings_position: {missing_before_pos:,} → {missing_after_pos:,} missing (calculated from constructor points)")

# Handle constructor rolling features (non-standings)
rolling_cols = [col for col in master.columns if any(x in col for x in 
    ['_last_', '_avg_', '_rate_', '_PRE_RACE', '_at_circuit', '_trend_'])]
constructor_rolling = [col for col in rolling_cols if 'constructor_' in col and 'standings' not in col.lower()]

if len(constructor_rolling) > 0:
    print("\nHandling constructor rolling features (non-standings):")
    print("Method: Adjacent constructor standings positions → forward-fill fallback")
    
    for col in constructor_rolling:
        if col in master.columns and master[col].dtype in [np.int64, np.float64]:
            # Step 1: Try adjacent constructor positions
            master, missing_before, missing_after_step1, filled_adjacent = impute_by_adjacent_constructor_position(
                master, col, race_col='raceId',
                constructor_pos_col='constructor_standings_position',
                max_offset=5
            )
            
            # Step 2: Forward-fill within constructor groups (if still missing)
            still_missing = master[col].isnull()
            if still_missing.sum() > 0:
                missing_before_step2 = still_missing.sum()
                master[col] = master.groupby('constructorId')[col].ffill()
                missing_after_ffill = master[col].isnull().sum()
                filled_ffill = missing_before_step2 - missing_after_ffill
                
                # Step 3: Fill remaining with same-race median or 0
                still_missing = master[col].isnull()
                if still_missing.sum() > 0:
                    missing_before_step3 = still_missing.sum()
                    # Try same-race median first
                    for race_id in master[still_missing]['raceId'].unique():
                        race_mask = master['raceId'] == race_id
                        race_median = master[race_mask & master[col].notna()][col].median()
                        if pd.notna(race_median):
                            master.loc[race_mask & still_missing, col] = race_median
                    
                    # Fill any remaining with 0
                    master[col] = master[col].fillna(0)
                    missing_after = master[col].isnull().sum()
                    filled_step3 = missing_before_step3 - missing_after
                else:
                    missing_after = missing_after_ffill
                    filled_step3 = 0
            else:
                missing_after = missing_after_step1
                filled_ffill = 0
                filled_step3 = 0
            
            if missing_before > 0:
                methods_used = []
                if filled_adjacent > 0:
                    methods_used.append(f"adjacent constructor positions ({filled_adjacent:,} filled)")
                if filled_ffill > 0:
                    methods_used.append(f"forward-fill within constructor ({filled_ffill:,} filled)")
                if filled_step3 > 0:
                    methods_used.append(f"same-race median/0 ({filled_step3:,} filled)")
                method_str = " → ".join(methods_used) if methods_used else "no change"
                print(f"  ✓ {col}: {missing_before:,} → {missing_after:,} missing ({method_str})")

print(f"\n✓ Constructor standings calculated and rolling features handled.")


CALCULATING CONSTRUCTOR STANDINGS
Method: Calculate from driver standings points (sum by constructor)

Calculating constructor_standings_points...

Calculating constructor_standings_position...

Handling constructor rolling features (non-standings):
Method: Adjacent constructor standings positions → forward-fill fallback

✓ Constructor standings calculated and rolling features handled.


## 13. Final Cleanup: Verify Data Completeness

After all imputation steps are complete, verify that the dataset is ready for modeling. This includes:
- Verifying that rows with missing rolling features were removed (done in Cell 16)
- Checking that calculated features (constructor standings) are complete
- Confirming data integrity

**Note:** Unnecessary columns were already removed early in Cell 12 to avoid wasting time imputing them.


In [52]:
# ========================================================================
# FINAL CLEANUP: Verify Data Completeness
# ========================================================================
print("="*70)
print("FINAL CLEANUP: VERIFY DATA COMPLETENESS")
print("="*70)

# Verify that rows with missing rolling features were removed (done in Cell 16)
print("\nVerifying that rows with missing rolling features were removed...")
features_requiring_history = [
    'driver_points_avg_last_10',
    'driver_podium_rate_last_10',
    'finished_lapped_rate_last_10',
    'dnf_rate_last_10',
    'disqualified_rate_last_10',
    'not_classified_rate_last_10',
    'driver_avg_position_last_5',
    'driver_avg_grid_last_5'
]

all_removed = True
for col in features_requiring_history:
    if col in master.columns:
        remaining_missing = master[col].isnull().sum()
        if remaining_missing > 0:
            print(f"  ⚠️ WARNING: {col} still has {remaining_missing:,} missing (should have been removed in Cell 16)")
            all_removed = False
        else:
            print(f"  ✓ {col}: No missing values (rows removed in Cell 16)")

if all_removed:
    print(f"\n  ✓ SUCCESS: All rows with missing rolling features were removed")
else:
    print(f"\n  ⚠️ WARNING: Some missing values remain - check Cell 16")

# Verify constructor standings (should be calculated, not imputed)
print("\nVerifying constructor standings (should be calculated, not imputed)...")
for col in ['constructor_standings_position', 'constructor_standings_points']:
    if col in master.columns:
        still_missing = master[col].isnull()
        if still_missing.sum() > 0:
            print(f"  ⚠️ {col}: {still_missing.sum():,} missing (should have been calculated)")
            print(f"     This indicates missing driver_standings_points - check calculation")
        else:
            print(f"  ✓ {col}: No missing values (calculated from driver standings)")

print("\n" + "="*70)
print("✓ Final cleanup complete!")
print("="*70)


FINAL CLEANUP: VERIFY DATA COMPLETENESS

Verifying that rows with missing rolling features were removed...
  ✓ driver_points_avg_last_10: No missing values (rows removed in Cell 16)
  ✓ driver_podium_rate_last_10: No missing values (rows removed in Cell 16)
  ✓ finished_lapped_rate_last_10: No missing values (rows removed in Cell 16)
  ✓ dnf_rate_last_10: No missing values (rows removed in Cell 16)
  ✓ disqualified_rate_last_10: No missing values (rows removed in Cell 16)
  ✓ not_classified_rate_last_10: No missing values (rows removed in Cell 16)
  ✓ driver_avg_position_last_5: No missing values (rows removed in Cell 16)
  ✓ driver_avg_grid_last_5: No missing values (rows removed in Cell 16)

  ✓ SUCCESS: All rows with missing rolling features were removed

Verifying constructor standings (should be calculated, not imputed)...
  ✓ constructor_standings_position: No missing values (calculated from driver standings)
  ✓ constructor_standings_points: No missing values (calculated from dr

## 14. Final Data Completeness Check

Perform a comprehensive final check to ensure all missing values have been appropriately handled:
- Identify any remaining data gaps (should be minimal)
- Separate legitimate structural missingness (Q2/Q3 for eliminated drivers) from actual data gaps
- Verify important features are complete
- Provide summary statistics

**Expected outcome:** Only Q2 and Q3 should have missing values (for drivers who didn't progress), and all other features should be complete.


In [53]:
# Final verification
final_missing = master.isnull().sum()
final_missing_pct = (final_missing / len(master)) * 100
final_missing_df = pd.DataFrame({
    'column': final_missing.index,
    'missing_count': final_missing.values,
    'missing_pct': final_missing_pct.values
}).sort_values('missing_pct', ascending=False)

print("="*70)
print("FINAL DATA COMPLETENESS CHECK")
print("="*70)

# Exclude target and date from missing check
cols_to_check = [col for col in master.columns if col not in ['podium', 'date']]
remaining_missing = final_missing_df[final_missing_df['column'].isin(cols_to_check) & 
                                     (final_missing_df['missing_pct'] > 0)]

# Separate legitimate structural missingness from data gaps
legitimate_missing = []  # Q2/Q3 for eliminated drivers, etc.
data_gaps = []  # Should have been imputed

for _, row in remaining_missing.iterrows():
    col = row['column']
    # Q2 and Q3 may have legitimate missing for eliminated drivers
    if col in ['q2', 'q3']:
        # Check if missing is for drivers who didn't make it (grid > 15 for Q2, grid > 10 for Q3)
        if col == 'q2':
            should_have = (master['grid'] <= 15) & master['grid'].notna()
        else:  # q3
            should_have = (master['grid'] <= 10) & master['grid'].notna()
        
        missing_should_have = (should_have & master[col].isnull()).sum()
        missing_shouldnt_have = ((~should_have) & master[col].isnull()).sum()
        
        if missing_should_have > 0:
            data_gaps.append((col, missing_should_have, "Should have time but missing"))
        if missing_shouldnt_have > 0:
            legitimate_missing.append((col, missing_shouldnt_have, "Didn't progress (legitimate)"))
    else:
        data_gaps.append((col, row['missing_count'], "Data gap"))

if len(data_gaps) == 0 and len(legitimate_missing) == 0:
    print("\n✓ SUCCESS: All features have been imputed!")
    print("  No missing values remaining in feature columns.")
elif len(data_gaps) == 0:
    print("\n✓ SUCCESS: All data gaps have been imputed!")
    print(f"  Remaining missing values are legitimate structural missingness:")
    for col, count, reason in legitimate_missing:
        print(f"    - {col}: {count:,} ({reason})")
else:
    print(f"\n⚠️  WARNING: {len(data_gaps)} features still have data gaps:")
    for col, count, reason in data_gaps:
        print(f"    - {col}: {count:,} missing ({reason})")
    
    if legitimate_missing:
        print(f"\n  Legitimate structural missingness:")
        for col, count, reason in legitimate_missing:
            print(f"    - {col}: {count:,} ({reason})")

# Summary statistics
print(f"\nDataset Statistics:")
print(f"  Total rows: {master.shape[0]:,}")
print(f"  Total columns: {master.shape[1]}")
print(f"  Total missing values: {master.isnull().sum().sum():,}")
print(f"  Columns with missing: {(final_missing > 0).sum()}")

# Check specific important columns
important_cols = ['grid', 'points', 'driverId', 'constructorId', 'year', 'circuitId',
                  'driver_podium_rate_last_10', 'driver_avg_grid_last_5', 
                  'constructor_podium_rate_last_15', 'driver_points_avg_last_10',
                  'q1', 'q2', 'q3']
print(f"\nImportant Features Missing Check:")
for col in important_cols:
    if col in master.columns:
        missing = master[col].isnull().sum()
        status = "✓" if missing == 0 else "⚠️"
        print(f"  {status} {col:35s}: {missing:6,} missing ({missing/len(master)*100:.2f}%)")

# Detailed Q1/Q2/Q3 analysis
print(f"\nQualifying Times Analysis:")
for col in ['q1', 'q2', 'q3']:
    if col in master.columns:
        total_missing = master[col].isnull().sum()
        if col == 'q1':
            print(f"  {col}: {total_missing:,} missing (all drivers should have Q1)")
        elif col == 'q2':
            should_have = (master['grid'] <= 15) & master['grid'].notna()
            missing_should = (should_have & master[col].isnull()).sum()
            print(f"  {col}: {total_missing:,} total missing, {missing_should:,} should have time")
        elif col == 'q3':
            should_have = (master['grid'] <= 10) & master['grid'].notna()
            missing_should = (should_have & master[col].isnull()).sum()
            print(f"  {col}: {total_missing:,} total missing, {missing_should:,} should have time")


FINAL DATA COMPLETENESS CHECK

✓ SUCCESS: All data gaps have been imputed!
  Remaining missing values are legitimate structural missingness:
    - q3: 6,445 (Didn't progress (legitimate))
    - q2: 3,572 (Didn't progress (legitimate))

Dataset Statistics:
  Total rows: 12,330
  Total columns: 61
  Total missing values: 10,017
  Columns with missing: 2

Important Features Missing Check:
  ✓ grid                               :      0 missing (0.00%)
  ✓ points                             :      0 missing (0.00%)
  ✓ driverId                           :      0 missing (0.00%)
  ✓ constructorId                      :      0 missing (0.00%)
  ✓ year                               :      0 missing (0.00%)
  ✓ circuitId                          :      0 missing (0.00%)
  ✓ driver_podium_rate_last_10         :      0 missing (0.00%)
  ✓ driver_avg_grid_last_5             :      0 missing (0.00%)
  ✓ constructor_podium_rate_last_15    :      0 missing (0.00%)
  ✓ driver_points_avg_last_10      

# 15. Imputation Methods Summary

## Placeholder Features
**Method:** Removal  
**Rationale:** 100% missing - FastF1 data not extracted yet. No information value.  
**Features:** `lap_time_variance`, `throttle_variance`, `overtake_attempts`, `avg_pit_stops`

## Sprint Features
**Method:** Impute with 0  
**Rationale:** Structural missingness - sprint races only exist 2021+. No sprint = 0 points.  
**Features:** `driver_sprint_points_avg_last_5`, `sprint_results_*`

## Qualifying Times (Q1/Q2/Q3)
**Method:** Track-specific imputation (remove invalid times first, then previous race at same track)  
**Rationale:** 1) Remove invalid times (Q2 for grid > 15, Q3 for grid > 10). 2) Track-specific: same-race adjacent grid → previous race at same track → fallbacks. Preserves structural missingness for eliminated drivers.  
**Details:** Q1: Impute all missing. Q2: Only for grid <= 15. Q3: Only for grid <= 10. Text times converted to seconds for imputation, then back to text format.  
**Features:** `q1`, `q2`, `q3`

## Other Qualifying Features
**Method:** Same-race, adjacent grid positions  
**Rationale:** Same race context. Adjacent grid positions reflect similar qualifying performance.  
**Features:** `qualifyId`, `qualifying_position`, `qualifying_*`

## Driver Standings Features
**Method:** Calculated from cumulative race points (not imputed)  
**Rationale:** Driver standings are calculated, not imputed. `driver_standings_points` = cumulative sum of `points` per driver. `driver_standings_position` = rank by cumulative points at each race.  
**Note:** `wins` column removed. `_PRE_RACE` versions created in 03.5 for modeling (shifted to avoid data leakage).  
**Features:** `driver_standings_points`, `driver_standings_position`

## Constructor Standings Features
**Method:** Calculated from driver standings points (not imputed)  
**Rationale:** Constructor standings are calculated, not imputed. `constructor_standings_points` = sum of `driver_standings_points` for all drivers in that constructor at that race. `constructor_standings_position` = rank of constructors by total points at that race.  
**Note:** `constructor_results_points` removed (redundant with cumulative standings). `_PRE_RACE` versions created in 03.5 for modeling.  
**Features:** `constructor_standings_points`, `constructor_standings_position`

## Circuit Podium Rates
**Method:** Fill with 0  
**Rationale:** No prior races at circuit = 0% podium rate (accurate representation).  
**Features:** `*_podium_rate_at_circuit`

## Driver Rolling Rate Features
**Method:** Rows removed (first races have no history)  
**Rationale:** These features require historical data and cannot be calculated for a driver's first race(s). Rows with missing values are removed rather than imputed to preserve data integrity.  
**Features:** `dnf_rate_last_10`, `finished_lapped_rate_last_10`, `disqualified_rate_last_10`, `not_classified_rate_last_10`, `driver_podium_rate_last_10`

## Driver Rolling Average Features
**Method:** Rows removed (first races have no history)  
**Rationale:** These features require historical data and cannot be calculated for a driver's first race(s). Rows with missing values are removed rather than imputed to preserve data integrity.  
**Features:** `driver_points_avg_last_10`, `driver_avg_position_last_5`, `driver_avg_grid_last_5`

## Constructor Rolling Features
**Method:** Adjacent constructor standings positions → forward-fill → same-race median/0  
**Rationale:** Same-race context first, then temporal, then same-race defaults.  
**Features:** `constructor_*_last_*`

## positionOrder_trend_season
**Method:** First race = 0 → forward-fill within driver-season → 0  
**Rationale:** First race has no trend (0). Forward-fill respects temporal order within season.  
**Features:** `positionOrder_trend_season`

---

## Key Principles

1. **Same-race imputation** (respects circuit, conditions, day)
2. **Adjacent position matching** (similar performance levels)
3. **Preserve legitimate missingness** (Q2/Q3 for eliminated drivers)
4. **Text lap times:** Convert to seconds → impute → convert back
5. **Domain-appropriate methods** for each feature type
6. **No forward fill across races** (each race is independent)
7. **Remove rows with missing rolling features** (first races have no history - cannot be imputed)
8. **Calculate, don't impute** (standings features are calculated from race results)

## 16. Save Final Cleaned Dataset

Save the dataset with all missing values appropriately handled and unnecessary columns removed, ready for modeling.


In [54]:
# Save final cleaned dataset
output_path = PROCESSED_ROOT / "master_races_clean.csv"
master.to_csv(output_path, index=False)
print(f"Final cleaned dataset saved to: {output_path}")
print(f"  Rows: {master.shape[0]:,}")
print(f"  Columns: {master.shape[1]}")
print(f"  File size: {output_path.stat().st_size / 1024**2:.2f} MB")
print(f"  Missing values: {master.isnull().sum().sum():,}")

# Compare with original refined dataset
refined_original = pd.read_csv(refined_file)
print(f"\nComparison with refined dataset:")
print(f"  Refined:  {refined_original.shape[0]:,} rows × {refined_original.shape[1]} columns")
print(f"  Clean:    {master.shape[0]:,} rows × {master.shape[1]} columns")
print(f"  Missing (refined): {refined_original.isnull().sum().sum():,}")
print(f"  Missing (clean):   {master.isnull().sum().sum():,}")
print(f"  Missing reduction: {refined_original.isnull().sum().sum() - master.isnull().sum().sum():,}")

print(f"\n✓ Dataset ready for modeling!")


Final cleaned dataset saved to: C:\Users\erikv\Downloads\F1\data\processed\master_races_clean.csv
  Rows: 12,330
  Columns: 61
  File size: 3.89 MB
  Missing values: 10,017

Comparison with refined dataset:
  Refined:  12,358 rows × 100 columns
  Clean:    12,330 rows × 61 columns
  Missing (refined): 252,341
  Missing (clean):   10,017
  Missing reduction: 242,324

✓ Dataset ready for modeling!
